In [10]:
from scipy import stats
import numpy as np

По данным опроса, 75% работников ресторанов утверждают, что испытывают на работе существенный стресс, оказывающий негативное влияние на их личную жизнь. Крупная ресторанная сеть опрашивает 100 своих работников, чтобы выяснить, отличается ли уровень стресса работников в их ресторанах от среднего. 67 из 100 работников отметили высокий уровень стресса.

Посчитайте достигаемый уровень значимости, округлите ответ до четырёх знаков после десятичной точки.

In [189]:
stats.binom_test(67, 100, 0.75, alternative = 'less')

0.044596325212681504

In [49]:
p1 = 0.67
p2 = 0.75
alpha = 0.210743   
z = stats.norm.ppf(1 - alpha / 2.)
left_boundary = (p2 - p1) - z * np.sqrt(p1 * (1 - p1)/100 + p2 * (1 - p2)/100)
right_boundary = (p2 - p1) + z * np.sqrt(p1 * (1 - p1)/100 + p2 * (1 - p2)/100)
print "Доверительный интервал: [%f, %f]" % (left_boundary, right_boundary)

Доверительный интервал: [0.000000, 0.160000]


The Wage Tract — заповедник в округе Тома, Джорджия, США, деревья в котором не затронуты деятельностью человека со времён первых поселенцев. Для участка заповедника размером 200х200 м имеется информация о координатах сосен (sn — координата в направлении север-юг, we — в направлении запад-восток, обе от 0 до 200).
pines.txt

Проверим, можно ли пространственное распределение сосен считать равномерным, или они растут кластерами.

Загрузите данные, поделите участок на 5х5 одинаковых квадратов размера 40x40 м, посчитайте количество сосен в каждом квадрате (чтобы получить такой же результат, как у нас, используйте функцию scipy.stats.binned_statistic_2d).

Если сосны действительно растут равномерно, какое среднее ожидаемое количество сосен в каждом квадрате? В правильном ответе два знака после десятичной точки.

In [51]:
import pandas as pd

In [84]:
data = pd.read_csv('pines.csv', header=0, sep='\t')

In [85]:
data.head()

,sn,we
0,200.0,8.8
1,199.3,10.0
2,193.6,22.4
3,167.7,35.6
4,183.9,45.4


In [86]:
len(data)

584

In [113]:
expected_quantity = (len(data)/(200.0*200))*40*40
print "Среднее ожидаемое количество сосен:", expected_quantity

Среднее ожидаемое количество сосен: 23.36


In [179]:
expected_frequences = [expected_quantity for x in range(0, 25)]

In [172]:
expected_quantities = np.zeros((5, 5))
expected_quantities[:,:] = expected_quantity
expected_quantities = expected_quantities.reshape(25,1)
expected_quantities

array([[ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36],
       [ 23.36]])

In [174]:
data['n_pines'] = 1
bins, xb, yb, binnum = stats.binned_statistic_2d(data.iloc[:,0],
                            data.iloc[:,1],
                            data.iloc[:,2],
                            statistic = 'sum',
                            bins=5)

In [180]:
observed_quantities = bins
observed_quantities = observed_quantities.reshape(1, 25)
observed_quantities

array([[ 18.,  26.,  29.,  25.,  20.,  14.,  12.,  22.,  34.,  25.,  12.,
         24.,  15.,  50.,  37.,   8.,  19.,  31.,  50.,   7.,   7.,   8.,
         37.,  27.,  27.]])

In [182]:
observed_frequences = []
for i in range(25):
    observed_frequences.append(observed_quantities[0, i])

In [183]:
observed_frequences

[18.0,
 26.0,
 29.0,
 25.0,
 20.0,
 14.0,
 12.0,
 22.0,
 34.0,
 25.0,
 12.0,
 24.0,
 15.0,
 50.0,
 37.0,
 8.0,
 19.0,
 31.0,
 50.0,
 7.0,
 7.0,
 8.0,
 37.0,
 27.0,
 27.0]

Чтобы сравнить распределение сосен с равномерным, посчитайте значение статистики хи-квадрат для полученных 5х5 квадратов. Округлите ответ до двух знаков после десятичной точки.

In [184]:
stats.chisquare(observed_frequences, expected_frequences, ddof = 1)

Power_divergenceResult(statistic=150.58904109589042, pvalue=9.8780617882088893e-21)

Насколько велико это значение? Если нулевая гипотеза справедлива, с какой вероятностью его можно было получить случайно?

Нулевое распределение статистики — хи-квадрат с 25−1=24 степенями свободы (поскольку у равномерного распределения, с которым мы сравниваем данные, нет ни одного оцениваемого по выборке параметра, число степеней свободы K−1, где K — количество интервалов).

Посчитайте достигаемый уровень значимости.

Если вы используете функцию scipy.stats.chi2.cdf, в качестве значения параметра df нужно взять 24 (это число степеней свободы); если функцию scipy.stats.chisquare — параметр ddof нужно брать равным 0 (это как раз количество параметров теоретического распределения, оцениваемых по выборке).

Отвергается ли гипотеза равномерности на уровне значимости 0.05?